#### Each panel has some dates missing: add those dates, for all hours

In [1]:
import pandas as pd
from checkMissing import checkMissing
from config import getClient
from os import listdir
from os.path import isfile, join
from datetime import datetime, timedelta

import time

In [2]:
dataloc = getClient("OutFront")

In [3]:
panels = [f for f in listdir(dataloc+"panels/") if isfile(join(dataloc+"panels/", f))]

print("{:,.0f} panels to process".format(len(panels)))

1,000 panels to process


In [4]:
# Use the population and hours from the week prior, or following,
# for this record
def getVals(df, d):    
    try:
        dt = d["date"] - timedelta(days=7)
        tmp = df.loc[d["panel"], dt]
    except:
        dt  = d["date"] + timedelta(days=7)        
        tmp = df.loc[d["panel"], dt]
    pop   = tmp["population"].reset_index(drop=True)
    hours = tmp["hour"].reset_index(drop=True)
    return pop, hours

In [5]:
# A panel is missing dates
def addDates(panel, missing):
    print(panel)
    input()
    d = {}
    d["panel"] = panel
    
    for dt in missing:
        d["date"]  = dt
        d["population"], d["hour"] = getVals(df, d)
        imputed = pd.DataFrame.from_dict(d)
        imputed = imputed.set_index(["panel", "date"])
        dfList.append(imputed)
    return dfList

In [6]:
def processPanel(df):
    missing = checkMissing(df, "day")
    if len(missing) > 0:
        df.set_index(["panel", "date"], inplace=True)
        dfList = addDates(df, missing)
        for x in dfList:
            df = df.append(x)
    return df

In [7]:
count = 0
start = time.time()

for panel in panels:
    count += 1
    if count %100 == 0: print(count)
    df = pd.read_csv(dataloc+"panels/"+panel,\
                     dtype={"hour":'int8',\
                            "population":'int32'})
    df = processPanel(df)
    df.to_csv(dataloc+"panels/"+panel, index=False)
    
end = time.time()
print("Completed after {:.0f} minutes".format((end-start)/60))

100
200
300
400
500
600
700
800
900
1000
Completed after 1 minutes
